In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt
%matplotlib inline

### In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:

#### Write a function to normalize features
#### Implement coordinate descent for LASSO
#### Explore effects of L1 penalty

In [508]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

#### Next, from Module 2 (Multiple Regression), copy and paste the ‘get_numpy_data’ function (or equivalent) that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. ‘price’). This function returns a ‘feature_matrix’ (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an ‘output_array’ which is an array of the values of the output in the data set (e.g. ‘price’).

In [509]:
def get_numpy_data(data_frame, features, output):
    """
    data_frame: pd.Dataframe
    featrues: a list of features name (e.g. [‘sqft_living’, ‘bedrooms’])
    output: a name of the output (e.g. ‘price’) 
    """
    # create a constant column with value one
    constant_column = np.ones((len(data_frame), 1))
    
    # create the features matrix
    feature_matrix = np.hstack((constant_column, data_frame.as_matrix(columns=features)))
    
    # this will convert the pd.Series into a numpy array
    output_name = [output]
    # as_matrix accept list as keywords
    output_array = data_frame.as_matrix(columns=output_name)
    
    return feature_matrix, output_array

#### Similarly, copy and paste the ‘predict_output’ function (or equivalent) from Module 2. This function accepts a 2D array ‘feature_matrix’ and a 1D array ‘weights’ and return a 1D array ‘predictions’.

In [510]:
def predict_output(feature_matrix, weights):
    return np.dot(feature_matrix, weights)

#### In the house dataset, features vary wildly in their relative magnitude: ‘sqft_living’ is very large overall compared to ‘bedrooms’, for instance. As a result, weight for ‘sqft_living’ would be much smaller than weight for ‘bedrooms’. This is problematic because “small” weights are dropped first as l1_penalty goes up.
#### To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

#### Write a short function called ‘normalize_features(feature_matrix)’, which normalizes columns of a given feature matrix. The function should return a pair ‘(normalized_features, norms)’, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.

In [511]:
def normalized_features(features):
    # np.linalg.norm(x, ord=None) when ord = None, it is 2-norm
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features / norms
    
    return normalized_features, norms
    

In [512]:
x = np.array([[3,6,9],[4,8,12]])

In [513]:
normalized_features(x)

(array([[0.6, 0.6, 0.6],
        [0.8, 0.8, 0.8]]), array([ 5., 10., 15.]))

In [437]:
x / np.sqrt(np.sum(x ** 2, axis=0))

array([[0.6, 0.6, 0.6],
       [0.8, 0.8, 0.8]])

### Review of Coordinate Descent

#### We seek to obtain a sparse set of weights by minimizing the LASSO cost function
#### SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
##### (By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

#### The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for
#### argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]

#### where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.
##### Pick a coordinate i
##### Compute w[i] that minimizes the LASSO cost function
##### Repeat the two steps for all coordinates, multiple times

#### For this assignment, we use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:

![Week5_Assignment2_picture.png](Week5_Assignment2_picture.png)

### Effect of L1 penalty
#### Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.
##### First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
##### Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
##### Set initial weights to [1,4,1].
##### Make predictions with feature matrix and initial weights.
##### Compute values of ro[i], where

#### Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?
#### Quiz Question: What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate?

#### So we can say that ro[i] quantifies the significance of the i-th feature: the larger ro[i] is, the more likely it is for the i-th feature to be retained

In [514]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
train = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
features = ['sqft_living', 'bedrooms']
output = 'price'

In [125]:
feature_matrix, output_array = get_numpy_data(sales, features, output)

In [238]:
output_array.shape

(21613, 1)

In [121]:
simple_feature_matrix, norms = normalized_features(feature_matrix)

In [122]:
simple_feature_matrix, norms

(array([[0.00680209, 0.00353021, 0.00583571],
        [0.00680209, 0.00768869, 0.00583571],
        [0.00680209, 0.00230361, 0.00389048],
        ...,
        [0.00680209, 0.00305154, 0.00389048],
        [0.00680209, 0.00478673, 0.00583571],
        [0.00680209, 0.00305154, 0.00389048]]),
 array([1.47013605e+02, 3.34257264e+05, 5.14075870e+02]))

In [13]:
weights = np.array([[1], [4], [1]])

In [14]:
predictions = predict_output(simple_feature_matrix, weights)

In [439]:
predictions.shape

(21613, 1)

In [442]:
(output_array - predictions + weights[1] * 
               simple_feature_matrix[:, 1].reshape(len(simple_feature_matrix), 1))

array([[221899.98736219],
       [537999.98736219],
       [179999.98930743],
       ...,
       [402100.98930743],
       [399999.98736219],
       [324999.98930743]])

In [445]:
ro_1 = np.dot(simple_feature_matrix[:, 1].reshape(1, len(simple_feature_matrix)), 
              (output_array - predictions + weights[1] * 
               simple_feature_matrix[:, 1].reshape(len(simple_feature_matrix), 1)))

ro_2 = np.dot(simple_feature_matrix[:, 2].reshape(1, len(simple_feature_matrix)), 
              (output_array - predictions + weights[2] * 
               simple_feature_matrix[:, 2].reshape(len(simple_feature_matrix), 1)))

In [446]:
print(ro_1, ro_2)

[[87939470.8232518]] [[80966698.66623941]]


![Week5_Assignment2_picture.png](Week5_Assignment2_picture.png)

In [708]:
np.array(87939470.8232518 * 2)

array(1.75878942e+08)

In [709]:
np.array(80966698.66623941 * 2)

array(1.61933397e+08)

#### Quiz Question: Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?
#### Quiz Question: What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were to take a step in that coordinate?

In [17]:
range_l1penalty = [2 * ro_2, 2 * ro_1]

In [18]:
range_l1penalty

[array([1.61933397e+08]), array([1.75878942e+08])]

### Single Coordinate Descent Step
#### Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [464]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot(feature_matrix[:, i].reshape(1, len(feature_matrix)), 
              (output - prediction + weights[i] * 
               feature_matrix[:, i].reshape(len(feature_matrix), 1)))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    
    elif ro_i < -l1_penalty / 2.:
        
        new_weight_i = ro_i + l1_penalty / 2
        
    elif ro_i > l1_penalty / 2.:
        new_weight_i = ro_i - l1_penalty / 2
        
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [468]:
lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                                           [2./math.sqrt(13),3./math.sqrt(10)]]), 
                              np.array([1., 1.]).reshape(2, 1), np.array([1., 4.]).reshape(2, 1), 0.1)

array([[0.42555885]])

### Cyclical coordinate descent

#### Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.
#### When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop

#### For each iteration:

###### As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
###### After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step

In [470]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    max_change = tolerance * 2
    
    weights = initial_weights
    
    while max_change > tolerance:
        max_change = 0
        
        for i in range(len(weights)):
            old_weight_i = weights[i].copy()
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            
            change = np.abs(weights[i] - old_weight_i)

            if change > max_change:
                
                max_change = change
        #print("{} Done".format(i))
    return weights

In [453]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3).reshape(3, 1)
l1_penalty = 1e7
tolerance = 1.0

#### Quiz Question: What is the RSS of the learned model on the normalized dataset?

In [282]:
my_simple_feature, my_simple_output = get_numpy_data(sales, simple_features, my_output)
my_simple_normalized_matrix, my_simple_norms = normalized_features(feature_matrix)

In [475]:
new_weights = lasso_cyclical_coordinate_descent(my_simple_normalized_matrix, my_simple_output, 
                                  initial_weights, l1_penalty, tolerance)

In [479]:
new_weights

array([[21624996.13459096],
       [63157248.87730661],
       [       0.        ]])

In [478]:
np.sum((my_simple_output - predict_output(my_simple_normalized_matrix, new_weights)) ** 2)

1630492460021214.0

#### Evaluating LASSO fit with more features

##### 17. Let us split the sales dataset into training and test sets. If you are using GraphLab Create, call ‘random_split’ with .8 ratio and seed=0. Otherwise, please down the corresponding csv files from the downloads section.
##### 18. Create a normalized feature matrix from the TRAINING data with the following set of features.
###### bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated

In [535]:
my_train_features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view',
                  'condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated']

In [536]:
my_output = 'price'

In [537]:
train_feature_matrix, output_array = get_numpy_data(train, my_train_features, my_output)

In [548]:
train_normalized_features_matrix, normalization_value = normalized_features(train_feature_matrix.astype(float))

In [547]:
np.linalg.norm(train_feature_matrix.astype(float), axis=0)# == np.sum(train_feature_matrix.astype(float) ** 2, axis=0)

array([9.87977733e+01, 3.46770818e+02, 2.22709592e+02, 2.25597973e+05,
       4.34516277e+06, 1.55954320e+02, 9.05538514e+00, 8.16026960e+01,
       3.43512736e+02, 7.65904694e+02, 1.95467912e+05, 5.24647562e+04,
       1.94732030e+05, 4.09449564e+04])

In [687]:
T = np.sqrt(np.sum(train_feature_matrix.astype(float) ** 2, axis=0))

#### l1_penalty is 1e7
#### Quiz Question: What features had non-zero weight in this case?

In [659]:
l1_penalty = 1e7
my_initial_weights = np.zeros((len(my_train_features) + 1, 1))#.reshape(len(my_train_features) + 1, 1)
my_tolerance = 1

In [666]:
my_weight_1e7 = lasso_cyclical_coordinate_descent(train_normalized_features_matrix, output_array,
                                              np.zeros((len(my_train_features) + 1, 1)), 1e7, 1)

In [668]:
pd.DataFrame(my_weight_1e7, index=['intercept'] + my_train_features, columns=['Coef Value'])

,Coef Value
intercept,2.386469e+07
bedrooms,0.000000e+00
bathrooms,0.000000e+00
sqft_living,3.049555e+07
sqft_lot,0.000000e+00
floors,0.000000e+00
waterfront,1.901634e+06
view,5.705765e+06
condition,0.000000e+00
grade,0.000000e+00


#### l1_penalty is 1e8
#### Quiz Question: What features had non-zero weight in this case?

In [669]:
my_weight_1e8 = lasso_cyclical_coordinate_descent(train_normalized_features_matrix, output_array,
                                              np.zeros((len(my_train_features) + 1, 1)), 1e8, 1)

In [694]:
pd.DataFrame(my_weight_1e8, index=['intercept'] + my_train_features, columns=['Coef Value'])

,Coef Value
intercept,5.362100e+07
bedrooms,0.000000e+00
bathrooms,0.000000e+00
sqft_living,0.000000e+00
sqft_lot,0.000000e+00
floors,0.000000e+00
waterfront,0.000000e+00
view,0.000000e+00
condition,0.000000e+00
grade,0.000000e+00


#### l1_penalty is 1e4
#### Quiz Question: What features had non-zero weight in this case?

In [672]:
my_weight_1e4 = lasso_cyclical_coordinate_descent(train_normalized_features_matrix, output_array,
                                              np.zeros((len(my_train_features) + 1, 1)), 1e4, 5e5)

In [673]:
pd.DataFrame(my_weight_1e4, index=['intercept'] + my_train_features, columns=['Coef Value'])

,Coef Value
intercept,5.748109e+07
bedrooms,-1.365263e+07
bathrooms,1.246271e+07
sqft_living,5.794279e+07
sqft_lot,-1.475770e+06
floors,-4.904548e+06
waterfront,5.349050e+06
view,5.845254e+06
condition,-4.160390e+05
grade,2.682275e+06


### Rescaling learned weights

#### Recall that we normalized our feature matrix, before learning the weights. To use these weights on a test set, we must normalize the test data in the same way. Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:
##### In this case, we must scale the resulting weights so that we can make predictions with original features:
##### Store the norms of the original features to a vector called ‘norms’:
##### Run Lasso on the normalized features and obtain a ‘weights’ vector
##### Compute the weights for the original features by performing element-wise division, i.e.
##### weights_normalized = weights / norms

In [698]:
weightsle7_normalized = my_weight_1e7 / normalization_value.reshape(14, 1)

In [696]:
weightsle8_normalized = my_weight_1e8 / normalization_value.reshape(14, 1)
weightsle4_normalized = my_weight_1e4 / normalization_value.reshape(14, 1)

### Evaluating each of the learned models on the test data

In [689]:
test_feature_matrix, test_output = get_numpy_data(test, my_train_features, 'price')

In [700]:
print('RSS on Test of penalty: 1e4 is: ', 
      np.sum((test_output - predict_output(test_feature_matrix.astype(float), weightsle4_normalized)) ** 2))

RSS on Test of penalty: 1e4 is:  129085259901909.12


In [701]:
print('RSS on Test of penalty: 1e7 is: ', 
      np.sum((test_output - predict_output(test_feature_matrix.astype(float), weightsle7_normalized)) ** 2))

RSS on Test of penalty: 1e7 is:  163103564164999.12


In [703]:
print('RSS on Test of penalty: 1e8 is: ', 
      np.sum((test_output - predict_output(test_feature_matrix.astype(float), weightsle8_normalized)) ** 2))

RSS on Test of penalty: 1e8 is:  284718925209874.0
